# 会话数据清洗及分析
* 参考示例数据结构

## 1.准备数据

In [1]:
import pymongo
# MongoDB连接配置
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
convoinsight_db = mongo_client["convoinsight"]
conversations_collection = convoinsight_db["conversations"]

In [2]:
import pandas as pd
data = pd.read_excel("./体验家-用户原声-new.xlsx")

In [3]:
data.head()

,会话内容
0,系统: 有新的咨询进来了。\n 沐沐: 您好，欢迎光临影石insta360~我是沐沐，请问有...
1,系统: 有新的咨询进来了。\n 金鱼: 您好，感谢您联系影石Insta360，我是金鱼，很高...
2,系统: 有新的咨询进来了。\n 树叶: 您好，欢迎光临影石insta360~我是树叶，请问有...
3,系统: 有新的咨询进来了。\n 柚子: 您好，感谢您联系影石Insta360，我是柚子，很高...
4,系统: 有新的咨询进来了。\n 15219877537: 手机没办法升级固件怎么办\n 糖米...


In [4]:
data['会话内容'][0]

'系统: 有新的咨询进来了。\n 沐沐: 您好，欢迎光临影石insta360~我是沐沐，请问有什么可以帮到您的吗？\n 15737314172: 安卓手机连接云台吗\n 沐沐: 请问您的手机型号是什么呢\n 15737314172: 荣耀pro\n 15737314172: 荣耀100pro\n 沐沐: 沐沐帮您查看一下，请您稍等下。\n 沐沐: 感谢您的耐心等待，您可以点此链接查看已经测试过可以连接云台并且能够使用功能的机型。\n 值得说明的是，由于安卓/iOS机型众多，以上兼容列表只是包含我们实测过的机型。对列表以外手机，一般只要能安装APP都可以使用，但可能由于性能原因个别功能会有限制，如4K录像、美颜等，球形全景等。感谢您的理解～ \n 温馨提示：除了手机系统需要兼容外，Flow/Flow Pro对手机的尺寸和重量也会有一定的要求哦，适配手机厚度：6.9~10mm；适配手机宽度：64~84mm；适配手机重量：＜300克。\n 沐沐: https://s2-cs-pub-std.kefutoutiao.com/doc/im/tid16480/Flow_Pro兼容性列表1735539287761.pdf\n 15737314172: https://s2-cs-pub-std.kefutoutiao.com/chat_upload/tid16480/1000043380_1735539423_1735539423379_e54a0c.jpg?OSSAccessKeyId=LTAI5tERQ5hZtMsgME6Q1Uap&Expires=1735568256&Signature=zFOL8W99%2FXL%2FUUzYNxUi6y3UHwA%3D\n 15737314172: 荣耀只有这两个可以吗\n '

## 2. 调用代理

In [5]:
import os
# Set the proxy URL and port
proxy_url = 'http://127.0.0.1'
proxy_port = "3566" # !!!please replace it with your own port

# Set the http_proxy and https_proxy environment variables
os.environ['http_proxy'] = f'{proxy_url}:{proxy_port}'
os.environ['https_proxy'] = f'{proxy_url}:{proxy_port}'

## 3. 单条数据:抽取会话内容的基础信息

In [26]:
def system_prompt_for_info_extraction():
    prompt = """请将以下客服会话数据转换成结构化的JSON格式。根据对话内容提取相关信息，并按照以下结构组织：

1. 生成唯一会话ID（如果原始数据中没有明确ID，可使用时间戳或随机字符串）
2. 确定客服人员姓名
3. 提取客户信息（手机号或用户ID）
4. 分析互动情况（总消息数、客服消息数、客户消息数）
5. 按顺序整理所有消息，包括:
   - 消息类型（系统/客服/客户）
   - 消息内容
   - 发送者
   - 对客户消息添加情感分析（正向/中立/负向）

请使用以下JSON结构：

{
  "agent": "客服姓名",
  "customerInfo": {
    "userId": "客户ID或手机号",
    "device": "相关设备信息（如有）",
    "history": "客户历史信息（如有）"
  },
  "interactionAnalysis": {
    "totalMessages": 消息总数,
    "agentMessages": 客服消息数,
    "userMessages": 客户消息数
  },
  "messages": [
    {
      "type": "消息类型（system/agent/user）",
      "content": "消息内容",
      "sender": "发送者名称/ID",
      "sentiment": "情感分析（仅用户消息需要）"
    },
    // 更多消息...
  ]
}    
    """
    return prompt

def user_prompt_for_info_extraction(conversations_string):
    prompt = """# 原始会话数据：
{conversations_string}
请根据上述要求将原始会话数据转换为结构化的JSON格式。
    """.format(conversations_string = conversations_string)
    return prompt

In [27]:
print(user_prompt_for_info_extraction("test"))

# 原始会话数据：
test
请根据上述要求将原始会话数据转换为结构化的JSON格式。
    


In [28]:
from openai import OpenAI
import os
# client = OpenAI(api_key=os.getenv('DEEPSEEK_API_KEY'), base_url="https://api.deepseek.com")
client = OpenAI(api_key=os.getenv('OPEN_AI_KEY'))
model_name = "gpt-4o-mini"

In [29]:
import uuid
from datetime import datetime
def custom_length_uuid(length=8):
    """Generate a UUID of custom length (minimum recommended: 8)"""
    # Generate a random UUID and remove hyphens
    u = str(uuid.uuid4()).replace('-', '')
    # Return a substring of the desired length
    return "#"+u[:length]


In [30]:
custom_length_uuid()

'#4966b00d'

In [31]:
import json
def process_single_conversation_extract(conversation:str,conversation_id:str,test_mode:bool,model="gpt-4o-mini"):
    """
    处理单条评论数据。

    参数:
        conversation: 客服会话对话信息
        test_mode: 是否为测试模式,布尔类型
        model:处理模型版本

    返回:
        Dict[str, Any]: 处理结果字典
    """
    
    try:
        # 准备提示文本
        system_prompt = system_prompt_for_info_extraction()
        user_prompt = user_prompt_for_info_extraction(conversations_string = conversation)

        # 调用模型获取结果
        response = client.chat.completions.create(
            model = model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt },
            ],
            stream=False,
                response_format={
                'type': 'json_object'
            }
        )

        # 获取 token 消耗数量
        token_usage = response.usage.total_tokens if hasattr(response, 'usage') else None

        try:
            content = response.choices[0].message.content
            parsed_content = json.loads(content)  # 解析内容
            # print(parsed_content)
            result_doc = parsed_content
            result_doc['origin_conversation'] = conversation
            result_doc['token_usage'] = token_usage
            result_doc['id'] = conversation_id
            result_doc['time'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        except json.JSONDecodeError as e:
            print(f"JSON 解析失败: {e}")
        except AttributeError as e:
            print(f"属性访问失败: {e}")
        except Exception as e:
            print(f"处理过程中发生错误: {e}")
        
        if test_mode:
            return result_doc
        else:
            conversations_collection.insert_one(result_doc)
            print(f"已插入一条数据到conversations集合")
            return result_doc

    except Exception as e:
        # 记录错误信息
        result_doc = {
            'error': str(e),
            "id":conversation_id,
            'processed_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'status': 'failed',
            "model": model,
        }
        return result_doc

In [32]:
response = process_single_conversation_extract(conversation=data['会话内容'][1],conversation_id="#001",test_mode=False)

已插入一条数据到conversations集合


In [33]:
response

{'agent': '金鱼',
 'customerInfo': {'userId': '18090636943', 'device': '未知', 'history': '无'},
 'interactionAnalysis': {'totalMessages': 9,
  'agentMessages': 5,
  'userMessages': 4},
 'messages': [{'type': 'system', 'content': '有新的咨询进来了。', 'sender': '系统'},
  {'type': 'agent',
   'content': '您好，感谢您联系影石Insta360，我是金鱼，很高兴为您服务！',
   'sender': '金鱼'},
  {'type': 'user',
   'content': '你好为什么我的acepro拍视频会闪烁啊',
   'sender': '18090636943',
   'sentiment': '负向'},
  {'type': 'agent', 'content': '请问您指的闪烁是屏幕闪烁，还是其他的呢', 'sender': '金鱼'},
  {'type': 'user',
   'content': '录出来的视频',
   'sender': '18090636943',
   'sentiment': '中立'},
  {'type': 'user',
   'content': '摇晃的时候灯光录出来是闪烁的',
   'sender': '18090636943',
   'sentiment': '负向'},
  {'type': 'agent',
   'content': '您好，感谢您的反馈！当相机开启“暗光增稳”时，为了减少运动模糊，相机会自动提高快门速度，抗闪烁功能也会随之失效。由于相机快门速度与电视、电脑或其他LED光源的刷新频率不一致，画面便会出现频闪。 如您在非夜间运动场景下拍摄，我们建议您可以关闭该功能，并根据环境灯光闪烁频率调整抗闪烁设置，可以改善频闪现象。',
   'sender': '金鱼'},
  {'type': 'user',
   'content': '哦哦',
   'sender': '18090636943',
   

## 4. 单条数据：抽取会话内容的诊断信息

In [15]:
def system_prompt_for_info_update():
    prompt = """# 客服对话结构化分析
你是一个有着十年专业经验的客服总监，现在需要基于客户对话进行会话内容的分析和诊断
## 任务描述
分析以下客服与客户的对话内容，将其转化为结构化数据，包括:
1. 客户满意度、问题解决程度、客服态度和安全风险评估等关键指标
2. 对话摘要和主要问题识别
3. 问题解决情况及解决方案
4. 相关标签和关键词提取
5. 客服改进建议

## 分析指南
1. **对话指标评估**:
   - 客户满意度(0-100): 根据客户回应的积极程度、问题解决情况评估
   - 问题解决程度(0-100): 根据客户问题是否得到全面解答评估
   - 客服态度(0-100): 根据客服回应的礼貌度、专业性、响应速度评估
   - 风险评估(0-100): 分析对话中可能存在的安全隐患，数值越高代表越安全
2. **对话摘要**:
   - 提取对话的主要主题和要点，用1-2句话概括
3. **对话详细分析**:
   - 识别客户的主要问题
   - 确定问题解决状态(已解决/部分解决/未解决/无法解决)
   - 概括客服提供的解决方案
4. **标签和关键词**:
   - 标签：根据对话内容提取最匹配的相关标签
   - 关键词：识别客户对话中出现的热门词汇或产品名称
5. **改进建议**:
   - 根据对话分析，提供2-4条客服可以改进的具体建议

## 输出格式
请按以下JSON格式输出分析结果:

```json
{
  "metrics": {
    "satisfaction": { "value": 0-100 },
    "resolution": { "value": 0-100 },
    "attitude": { "value": 0-100 },
    "risk": { "value": 0-100 }
  },
  "summary": "一句话总结对话内容",
  "conversationSummary": {
    "mainIssue": "客户主要问题",
    "resolutionStatus": {
      "status": "已解决/部分解决/未解决/无法解决",
      "description": "解决方案描述"
    },
    "mainSolution": "客服提供的主要解决方案"
  },
  "tags": ["标签1"],
  "improvementSuggestions": [
    "改进建议1",
    "改进建议2",
    "改进建议3"
  ],
  "hotWords": ["关键词1", "关键词2", "关键词3"]
}
```
## 特别注意事项
1. 所有评分必须基于客观事实，避免主观臆断
2. 标签应精准反映对话主题和产品类别
3. 改进建议应具体、可操作且有建设性
4. 如对话未明确表明某信息，应做合理推断并注明
    """
    return prompt

def user_prompt_for_info_update(conversations_string,tags_list_string):
    prompt = """# 原始会话数据：
{conversations_string}
请根据上述要求将原始会话数据转换为结构化的JSON格式。
# 标签的识别请从以下标签进行选择
{tags_list_string}
    """.format(conversations_string = conversations_string,tags_list_string=tags_list_string)
    return prompt

In [16]:
problem_tags_list = """产品使用咨询>功能使用
产品使用咨询>相机操作咨询
产品使用咨询>Insta360 APP咨询
产品使用咨询>Studio 咨询
产品使用咨询>教程需求咨询
产品使用咨询>配件使用
产品使用咨询>云服务使用咨询
产品使用咨询>Connect使用咨询
产品使用咨询>其他产品使用咨询
相机故障问题>镜头外观不良
相机故障问题>镜头功能类不良
相机故障问题>屏外观不良
相机故障问题>屏功能不良
相机故障问题>按键外观/功能不良
相机故障问题>电池外观(功能)不良
相机故障问题>USB盖/喇叭盖/其他盖子损坏
相机故障问题>机身外观不良
相机故障问题>进液
相机故障问题>充电类不良
相机故障问题>发热
相机故障问题>拼接类不良
相机故障问题>Wifi连接类不良
相机故障问题>蓝牙连接不良
相机故障问题>有线连接类不良
相机故障问题>声音类不良
相机故障问题>图像类不良（预览&成片）
相机故障问题>防抖类异常
相机故障问题>死机
相机故障问题>开关机类不良
相机故障问题>升级异常
相机故障问题>续航异常
相机故障问题>相机录制异常/停止录像
相机故障问题>相机文件异常
相机故障问题>内存卡相关异常
相机故障问题>直播异常-专业级
相机故障问题>其他故障
"""

In [68]:
def validate_data_structure(data):
    """验证数据结构是否符合预期"""
    required_keys = {
        "metrics": dict,
        "summary": str,
        "conversationSummary": dict,
        "tags": list,
        "improvementSuggestions": list,
        "hotWords": list
    }
    
    for key, expected_type in required_keys.items():
        if key not in data or not isinstance(data[key], expected_type):
            return False
    
    # 进一步验证嵌套结构
    metrics_keys = {"satisfaction", "resolution", "attitude", "risk"}
    if not all(k in data["metrics"] for k in metrics_keys):
        return False
    
    conversation_summary_keys = {"mainIssue", "resolutionStatus", "mainSolution"}
    if not all(k in data["conversationSummary"] for k in conversation_summary_keys):
        return False
    
    resolution_status_keys = {"status", "description"}
    if not all(k in data["conversationSummary"]["resolutionStatus"] for k in resolution_status_keys):
        return False
    
    return True

def update_conversation(conversation_id, new_data):
    # 验证数据结构
    if not validate_data_structure(new_data):
        print("数据结构不符合预期，更新操作已取消。")
        return None
    
    # 查找现有文档
    existing_doc = conversations_collection.find_one({'id': conversation_id})
    
    # 准备更新数据
    update_data = new_data.copy()
    
    if existing_doc:
        # 更新时间为最新时间
        update_data['times'] = new_data.get('time', existing_doc.get('time'))
        # 合并 token_usage
        if 'token_usage' in new_data and 'token_usage' in existing_doc:
            update_data['token_usage'] = existing_doc['token_usage'] + new_data['token_usage']

        # 合并其他字段，保留现有字段，添加新字段
        for key in existing_doc:
            if key not in update_data and key != '_id':
                update_data[key] = existing_doc[key]
    
    # 执行更新操作
    result = conversations_collection.update_one(
        {'id': conversation_id},
        {'$set': update_data},
        upsert=True  # 如果文档不存在，则创建新文档
    )
    print(f"已更新一条数据到conversations集合")
    return result

In [69]:
def process_single_conversation_update(conversation:str,conversation_id:str,test_mode:bool,model="gpt-4o-mini"):
    """
    处理单条评论数据。

    参数:
        conversation: 客服会话对话信息
        test_mode: 是否为测试模式,布尔类型
        model:处理模型版本

    返回:
        Dict[str, Any]: 处理结果字典
    """
    
    try:
        # 准备提示文本
        system_prompt = system_prompt_for_info_update()
        user_prompt = user_prompt_for_info_update(conversations_string = conversation,tags_list_string = problem_tags_list)

        # 调用模型获取结果
        response = client.chat.completions.create(
            model = model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt },
            ],
            stream=False,
                response_format={
                'type': 'json_object'
            }
        )

        # 获取 token 消耗数量
        token_usage = response.usage.total_tokens if hasattr(response, 'usage') else None

        try:
            content = response.choices[0].message.content
            parsed_content = json.loads(content)  # 解析内容
            # print(parsed_content)
            result_doc = parsed_content
            result_doc['token_usage'] = token_usage
            result_doc['time'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        except json.JSONDecodeError as e:
            print(f"JSON 解析失败: {e}")
        except AttributeError as e:
            print(f"属性访问失败: {e}")
        except Exception as e:
            print(f"处理过程中发生错误: {e}")
        
        if test_mode:
            return result_doc
        else:
            # 更新原始数据
            update_conversation(conversation_id, result_doc)
            return result_doc

    except Exception as e:
        # 记录错误信息
        result_doc = {
            'error': str(e),
            "conversation":conversation,
            'processed_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'status': 'failed',
            "model": model,
        }
        return result_doc

In [82]:
test = conversations_collection.find_one()

In [83]:
test

{'_id': ObjectId('67ceba7fb6cb42d2b375e45d'),
 'agent': '栋栋',
 'customerInfo': {'userId': '15951750186', 'device': None, 'history': None},
 'interactionAnalysis': {'totalMessages': 6,
  'agentMessages': 4,
  'userMessages': 3},
 'messages': [{'type': 'system', 'content': '有新的咨询进来了。', 'sender': '系统'},
  {'type': 'agent',
   'content': '您好，欢迎光临影石insta360~我是栋栋，请问有什么可以帮到您的吗？',
   'sender': '栋栋'},
  {'type': 'user',
   'content': 'X4误闯洗衣机，',
   'sender': '15951750186',
   'sentiment': '中立'},
  {'type': 'user',
   'content': '昨天按照客服说放进米缸中一天后看看',
   'sender': '15951750186',
   'sentiment': '中立'},
  {'type': 'user',
   'content': '现在无法开机',
   'sender': '15951750186',
   'sentiment': '负向'},
  {'type': 'agent',
   'content': '您好，您可以通过点击以下链接，再点击售后服务申请，发起维修申请。 https://www.insta360.com/cn/support/workorder/repair_nav （在App端打开可能会出现填写卡顿的问题，您可以复制该网址至浏览器打开填写） *产品在保修期内出现非人为原因导致问题的情况，我们将提供免费维修服务。 *如已过保修期，具体的维修费用需要您先寄回产品，我们将根据具体检测结果提供维修报价，您可通过https://www.insta360.com/cn/support/workorder/repair_nav至【我的售后工

In [70]:
response = process_single_conversation_update(conversation=,conversation_id=,test_mode= False,model="gpt-4o-mini")

已更新一条数据到conversations集合


In [71]:
response

{'metrics': {'satisfaction': {'value': 75},
  'resolution': {'value': 50},
  'attitude': {'value': 85},
  'risk': {'value': 90}},
 'summary': '客户询问影石insta360 One RS是否有新款，客服表示未收到相关通知，并建议关注官方平台。',
 'conversationSummary': {'mainIssue': '客户询问影石insta360 One RS的新款产品情况。',
  'resolutionStatus': {'status': '未解决',
   'description': '客服无法提供有关新品的确切信息，只能建议客户关注官方渠道。'},
  'mainSolution': '客服建议客户关注官方网站和微信服务号，以获取新产品公告。'},
 'tags': ['产品使用咨询>其他产品使用咨询'],
 'improvementSuggestions': ['提供更详细的产品发布周期或新品发布预告信息，以加强客户沟通。',
  '在客服对话中加入常见问题的快速解答，提高响应效率。',
  '加强对新品动态的内部信息共享，减少客服对客户提问的无解情况。',
  '优化关闭会话的提示，确保客户已解决问题后再关闭，使客户感受更好。'],
 'hotWords': ['影石insta360', 'One RS', '新款'],
 'token_usage': 1784,
 'time': '2025-03-10 18:53:50'}

In [72]:
# data['id'] = [custom_length_uuid() for _ in range(len(data))]

In [73]:
# data.to_excel('./data_pro.xlsx')

## 1. 会话数据处理

In [74]:
import os
# 获取系统的 CPU 核心数
cpu_count = os.cpu_count()
default_max_workers = cpu_count * 5  # 默认线程数
# 设置新的 max_workers，增加 1.2 倍的线程数并转换为整数
new_max_workers = int(default_max_workers * 1.3)

In [75]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List
from tqdm import tqdm

def multi_process_conversation_extraction(
    conversations: List[dict],  # 修改为字典列表以包含 conversation_id
    max_workers: int = 4,
    model="gpt-4o-mini",
    test_mode=True
) -> List[dict]:
    """
    并行处理评论并存储结果。

    参数:
        conversations: 包含会话数据和ID的字典列表
        max_workers: 最大线程数
        model: 使用的模型名称
        test_mode: 是否为测试模式
    返回:
        处理结果的列表
    """
    results = []
    try:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_conversation = {
                executor.submit(
                    process_single_conversation_extract, 
                    conversation['content'], 
                    conversation['id'],  # 传递 conversation_id
                    test_mode, 
                    model
                ): conversation 
                for conversation in conversations
            }
            
            for future in tqdm(
                as_completed(future_to_conversation), 
                total=len(conversations),
                desc="处理会话"
            ):
                try:
                    result = future.result()
                    results.append(result)
                except Exception as e:
                    conversation = future_to_conversation[future]
                    print(f"处理会话 {conversation['id']} 时发生错误: {str(e)}")
                    
    except Exception as e:
        print(f"批处理过程中发生错误: {str(e)}")
    
    return results

In [76]:
all_data = pd.read_excel("./data_pro.xlsx")

In [77]:
all_data.head()

,Unnamed: 0,会话内容,id
0,0,系统: 有新的咨询进来了。\n 沐沐: 您好，欢迎光临影石insta360~我是沐沐，请问有...,#779102a4
1,1,系统: 有新的咨询进来了。\n 金鱼: 您好，感谢您联系影石Insta360，我是金鱼，很高...,#8d1ed6c9
2,2,系统: 有新的咨询进来了。\n 树叶: 您好，欢迎光临影石insta360~我是树叶，请问有...,#ea993536
3,3,系统: 有新的咨询进来了。\n 柚子: 您好，感谢您联系影石Insta360，我是柚子，很高...,#9bddb36b
4,4,系统: 有新的咨询进来了。\n 15219877537: 手机没办法升级固件怎么办\n 糖米...,#055c76f6


In [78]:
try:
    # 读取 Excel 数据
    data = pd.read_excel("./data_pro.xlsx")
    # 获取已处理的会话 ID
    processed_ids = set(doc['id'] for doc in conversations_collection.find({}, {'id': 1}))
    # 筛选未处理的会话
    unprocessed_conversations = [
        {'id': row['id'], 'content': row['会话内容']}
        for _, row in data.iterrows()
        if row['id'] not in processed_ids
    ]
    print(f"未处理的会话数量: {len(unprocessed_conversations)}")
    
    results = multi_process_conversation_extraction(unprocessed_conversations,max_workers=new_max_workers,test_mode=False)
except Exception as e:
    print(f"主程序执行错误: {str(e)}")

未处理的会话数量: 0


处理会话: 0it [00:00, ?it/s]


## 2. 会话信息更新

In [79]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List
from tqdm import tqdm

def multi_process_conversation_update(
    conversations: List[dict],  # 修改为字典列表以包含 conversation_id
    max_workers: int = 4,
    model="gpt-4o-mini",
    test_mode=True
) -> List[dict]:
    """
    并行处理评论并存储结果。

    参数:
        conversations: 包含会话数据和ID的字典列表
        max_workers: 最大线程数
        model: 使用的模型名称
        test_mode: 是否为测试模式
    返回:
        处理结果的列表
    """
    results = []
    try:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_conversation = {
                executor.submit(
                    process_single_conversation_update, 
                    conversation['content'], 
                    conversation['id'],  # 传递 conversation_id
                    test_mode, 
                    model
                ): conversation 
                for conversation in conversations
            }
            
            for future in tqdm(
                as_completed(future_to_conversation), 
                total=len(conversations),
                desc="处理会话"
            ):
                try:
                    result = future.result()
                    results.append(result)
                except Exception as e:
                    conversation = future_to_conversation[future]
                    print(f"处理会话 {conversation['id']} 时发生错误: {str(e)}")
                    
    except Exception as e:
        print(f"批处理过程中发生错误: {str(e)}")
    
    return results

In [80]:
try:
    # 获取已处理的会话 ID（仅包含有 metrics 字段的文档）# 筛选未处理的会话
    unprocessed_data =  conversations_collection.find({"id": {"$exists": True},
                                                            "metrics": {"$exists": False}
                                                        })
    unprocessed_conversations = []
    for row in unprocessed_data:
        unprocessed_conversations.append({'id': row['id'], 'content': row['origin_conversation']})
    print(f"未处理的会话数量: {len(unprocessed_conversations)}")
    
    # results = multi_process_conversation_update(unprocessed_conversations[:5], max_workers=new_max_workers, test_mode=False)
except Exception as e:
    print(f"主程序执行错误: {str(e)}")

未处理的会话数量: 1996
